# Aprendizaje basado en instancias

## Procesado de los datos

El conjunto de datos sobre reconocimiento de vinos está incluido en *Scikit-learn*, se obtiene usando la función `load_wine` incluida en la librería `sklearn.datasets`. Este conjunto de datos contiene 178 ejemplos de distintas variedades de vino, con 13 características y tres clasificaciones posibles.

In [2]:
from sklearn.datasets import load_wine

wine = load_wine()

Este conjunto de datos es un diccionario con varios campos:
* `data`: Es el conjunto de datos, se trata de un array en el que cada componente es un array con las características de cada instancia.
* `target`: Es el conjunto de valores de clasificación para cada instancia. Es un array del mismo tamaño que `data`, en el que se indica el valor de clasificación de cada instancia, en el mismo orden en que éstas se encuentran en el array `data`.
* `DESCR`: Es una descripción del conjunto de datos.
* `target_names`: Es un array con los nombres de cada valor de clasificación.
* `feature_names`: Es un array con los nombres de cada característica.

Almacenamos los datos en las variables `X_data`, `y_data`, `X_names` e `y_names`.

In [3]:
X_data, y_data, X_names, y_names = \
    wine.data, wine.target, wine.feature_names, wine.target_names

## Contenido del ejercicio

El ejercicio consiste en
* Definir una función que repita el proceso de entrenamiento incrementando el valor del número de vecinos en el algoritmo **k**-*NN* desde 1 hasta 15, devolviendo el mejor valor obtenido para un modelo en el que se han fijado la distancia a considerar `p` y la ponderación de las clasificaciones `weights`.
* Construir dos modelos de decisión (para valores concretos de `p` y `weights`) con el mejor valor de `k` obtenido con la función pedida en el punto 1, a partir del conjunto de datos original sin normalizar.
* Construir dos modelos de decisión (para valores concretos de `p` y `weights`) con el mejor valor de `k` obtenido con la función pedida en el punto 1, a partir del conjunto de datos normalizado.
* Comparar los rendimientos obtenidos con los cuatro modelos construidos.

El **desarrollo tiene que estar razonado**, indicando en cada apartado qué se está haciendo, **demostrando así el conocimiento adquirido en este módulo**. ¿Qué conclusiones puedes sacar de lo aprendido sobre aprendizaje basado en instancias?

## Ejercicio 

En primer lugar vamos a dividir el dataset inicial en conjuntos de entrenamiento y de testeo, en este caso utilizando un tercio del conjunto para testeo.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(X_data,y_data,test_size = 0.33,
                   random_state=4861,stratify=y_data)

A continuación importamos el algoritmo K-NN y hacemos una comprobación inicial para ver que valor de distancia es más óptimo entre la Euclidea y la Manhattan.

In [6]:
from sklearn.neighbors import KNeighborsClassifier

In [7]:
knn1 = KNeighborsClassifier(n_neighbors = 5, p = 1)

In [8]:
knn1.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                     weights='uniform')

In [9]:
knn1.score(X_test, Y_test)

0.7457627118644068

In [10]:
knn2 = KNeighborsClassifier(n_neighbors = 5, p = 2)

In [11]:
knn2.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [12]:
knn2.score(X_test,Y_test)

0.6949152542372882

Podemos ver en este caso que la distancia Manhattan da un mejor resultado, así que la utilizaremos para la primera parte de la actividad.

### Parte 1

Definimos un método que recibe conjuntos de entrenamiento y de testeo y devuelve el mejor valor para K.

In [13]:
def findBestK(x_train, y_train, x_test, y_test):
    res = 0
    for i in range(1,16):
        knns = KNeighborsClassifier(n_neighbors = i, p = 1,weights='distance')
        knns.fit(x_train, y_train)
        score = knns.score(x_test, y_test)
        if(score>res):
            res = score
            res1 = i
    return res, res1

In [14]:
findBestK(X_train, Y_train, X_test, Y_test)

(0.864406779661017, 3)

Si el atributo weights se indica como 'distance' el valor óptimo de k pasa a ser 3

### Parte 2

Construimos los nuevos modelos con el valor de k obtenido anteriormente y variando p y weights

In [15]:
knn21 = KNeighborsClassifier(n_neighbors = 3, p = 1, weights='distance')
knn21.fit(X_train, Y_train)
knn21.score(X_test, Y_test)

0.864406779661017

In [16]:
knn22 = KNeighborsClassifier(n_neighbors = 3, p = 2)
knn22.fit(X_train, Y_train)
knn22.score(X_test, Y_test)

0.6779661016949152

### Parte 3

Ahora normalizamos los datos antes de darselos a los modelos

Para ello importamos la clase StandardScaler

In [17]:
from sklearn.preprocessing import StandardScaler

Ahora entrenamos el normalizador con el conjunto de entrenamiento

In [18]:
normalizador = StandardScaler().fit(X_train)

Una vez hecho eso normalizamos los conjuntos de utilizando el método transfotm del normalizador

In [19]:
Xn_train = normalizador.transform(X_train)
Xn_test = normalizador.transform(X_test)

Y ya con los nuevos conjuntos normalizados se los pasamos a los modelos

In [20]:
knn31 = KNeighborsClassifier(n_neighbors = 3, p = 1, weights='distance')
knn31.fit(Xn_train, Y_train)
knn31.score(Xn_test, Y_test)

0.9661016949152542

In [21]:
knn32 = KNeighborsClassifier(n_neighbors = 3, p = 2)
knn32.fit(Xn_train, Y_train)
knn32.score(Xn_test, Y_test)

0.9491525423728814

### Parte 4

In [22]:
print('El modelo 1 tiene un rendimiento de: ', knn21.score(X_test, Y_test))
print('El modelo 2 tiene un rendimiento de: ', knn22.score(X_test, Y_test))
print('El modelo 3 tiene un rendimiento de: ', knn31.score(Xn_test, Y_test))
print('El modelo 4 tiene un rendimiento de: ', knn32.score(Xn_test, Y_test))

El modelo 1 tiene un rendimiento de:  0.864406779661017
El modelo 2 tiene un rendimiento de:  0.6779661016949152
El modelo 3 tiene un rendimiento de:  0.9661016949152542
El modelo 4 tiene un rendimiento de:  0.9491525423728814


Como podemos observar, el modelo que ha dado un mejor rendimiento se trata del **modelo 3**, que utiliza los **conjuntos normalizados**, la **distancia Manhattan**, el parámetro __weights como _distance___ y **k = 3**.